In [ ]:
# Acessar spark-shell ustilizando o comanda abaixo

./bin/spark-shell --packages com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.7,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1


In [ ]:
# Neste passo iremos criar um Data Frame onde será armazenado as mensagens do Kafka
val kafkaDF = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", "localhost:9092").
option("subscribe","kafka-chat").
load()

In [ ]:
# convertendo as mensagens do kafka para String
val chatString = kafkaDF.selectExpr("CAST(value AS STRING)")


In [ ]:
import org.apache.spark.sql.types._

val schema = new StructType().
                    add("author",StringType).
                    add("message",StringType)

        
val messages = chatString.
                    select(from_json(col("value"), schema).
                    as("data")).
                    select("data.*")
    

In [ ]:
# Exibe os dados no console
val query = messages.writeStream.
                outputMode("append").
                format("console").
                start();

In [ ]:
# Configurações necessárias para  acessar um bucket no AWS

spark.sparkContext.hadoopConfiguration.set("fs.s3a.access.key", "key")
spark.sparkContext.hadoopConfiguration.set("fs.s3a.secret.key", "secret")
spark.sparkContext.hadoopConfiguration.set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.sparkContext.hadoopConfiguration.set("com.amazonaws.services.s3.enableV4", "true")
spark.sparkContext.hadoopConfiguration.set("fs.s3a.aws.credentials.provider","org.apache.hadoop.fs.s3a.BasicAWSCredentialsProvider")
spark.sparkContext.hadoopConfiguration.set("fs.s3a.endpoint", "s3.amazonaws.com")

In [ ]:
import org.apache.spark.sql.DataFrame;

val query1 = messages.
    writeStream.
    foreachBatch { (batchDF: DataFrame, batchId: Long) =>   
                    batchDF.write.mode("append").csv("s3a://buckt/csv/chat") 
                 }.start();